# Store Sales and Profit Analysis

This notebook performs a comprehensive analysis of the retail performance using the Superstore dataset. 
We will focus on Sales and Profits using Python and Plotly.

In [1]:
import sys
# Ensure nbformat is installed for Plotly rendering
!{sys.executable} -m pip install nbformat --quiet

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Set display options
pd.set_option('display.max_columns', None)


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. Data Preparation

In [2]:
# Load the dataset
try:
    df = pd.read_csv('Superstores.csv', encoding='windows-1252')
except UnicodeDecodeError:
    df = pd.read_csv('Superstores.csv', encoding='ISO-8859-1')

# Display first few rows
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [3]:
# Inspect data types and missing values
df.info()
print("\nMissing Values:\n", df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [4]:
# Convert Date columns to DateTime
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])

# Extract Month and Year for temporal analysis
df['Order Month'] = df['Order Date'].dt.to_period('M').astype(str)
df['Order Year'] = df['Order Date'].dt.year

print("Date columns converted successfully.")

Date columns converted successfully.


In [5]:
# Handling Missing Values and Outliers
# (Assuming minimal missing data based on standard superstore dataset, but good to handle)
df = df.dropna()

# Simple outlier check on Sales and Profit using describe
df[['Sales', 'Profit']].describe()

,Sales,Profit
count,9994.000000,9994.000000
mean,229.858001,28.656896
std,623.245101,234.260108
min,0.444000,-6599.978000
25%,17.280000,1.728750
50%,54.490000,8.666500
75%,209.940000,29.364000
max,22638.480000,8399.976000


## 2. Sales Analysis

In [6]:
# Temporal Sales Trends (Monthly)
monthly_sales = df.groupby('Order Month')['Sales'].sum().reset_index()

fig_sales_trend = px.line(monthly_sales, x='Order Month', y='Sales', 
                          title='Monthly Sales Trend', markers=True)
fig_sales_trend.show()

In [7]:
# Category and Sub-Category Analysis
cat_sales = df.groupby(['Category', 'Sub-Category'])['Sales'].sum().reset_index()

fig_cat_sales = px.bar(cat_sales, x='Sales', y='Sub-Category', color='Category',
                       orientation='h', title='Sales by Category and Sub-Category',
                       text='Sales')
fig_cat_sales.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig_cat_sales.update_layout(yaxis={'categoryorder':'total ascending'})
fig_cat_sales.show()

## 3. Profit Analysis

In [8]:
# Profit Trends (Monthly)
monthly_profit = df.groupby('Order Month')['Profit'].sum().reset_index()

fig_profit_trend = px.line(monthly_profit, x='Order Month', y='Profit', 
                           title='Monthly Profit Trend', markers=True,
                           color_discrete_sequence=['green'])
fig_profit_trend.show()

In [9]:
# Most Profitable Products (Top 10)
top_products_profit = df.groupby('Product Name')['Profit'].sum().nlargest(10).reset_index()

fig_top_profit = px.bar(top_products_profit, x='Profit', y='Product Name', 
                        orientation='h', title='Top 10 Most Profitable Products',
                        color='Profit', color_continuous_scale='Viridis')
fig_top_profit.update_layout(yaxis={'categoryorder':'total ascending'})
fig_top_profit.show()

In [10]:
# Segmentation Analysis: Profitability by Customer Segment
segment_profit = df.groupby('Segment')['Profit'].sum().reset_index()

fig_segment_profit = px.pie(segment_profit, values='Profit', names='Segment', 
                            title='Profit Distribution by Customer Segment',
                            hole=0.4)
fig_segment_profit.show()

## 4. Operational Insights

In [11]:
# Sales-to-Profit Ratios across Segments
segment_metrics = df.groupby('Segment')[['Sales', 'Profit']].sum().reset_index()
segment_metrics['Sales_to_Profit_Ratio'] = segment_metrics['Sales'] / segment_metrics['Profit']
segment_metrics['Profit_Margin_%'] = (segment_metrics['Profit'] / segment_metrics['Sales']) * 100

print("Operational Metrics by Segment:")
display(segment_metrics)

fig_metrics = px.bar(segment_metrics, x='Segment', y='Profit_Margin_%', 
                     title='Profit Margin % by Customer Segment', 
                     color='Segment')
fig_metrics.show()

Operational Metrics by Segment:


,Segment,Sales,Profit,Sales_to_Profit_Ratio,Profit_Margin_%
0,Consumer,1.161401e+06,134119.2092,8.659471,11.548050
1,Corporate,7.061464e+05,91979.1340,7.677245,13.025506
2,Home Office,4.296531e+05,60298.6785,7.125416,14.034269


In [12]:
# Scatter Plot: Sales vs Profit Analysis
# Identify products with high sales but low/negative profit
fig_scatter = px.scatter(df, x='Sales', y='Profit', color='Discount', 
                         size='Quantity', hover_data=['Product Name', 'Sub-Category'],
                         title='Sales vs Profit (Colored by Discount)',
                         color_continuous_scale='RdBu')
fig_scatter.show()